# 201521466 김남현


### GridSearchCV

In [17]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train',
#메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

newsgroups_test = fetch_20newsgroups(subset='test', 
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

vectorizer = TfidfVectorizer() #TFIDF 변환.
newsgroups_train_tfidf = vectorizer.fit_transform(newsgroups_train.data)
newsgroups_test_tfidf = vectorizer.transform(newsgroups_test.data)

In [4]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
print("Parameter grid:\n{}".format(param_grid)) 
#파라미터 값 지정.

Parameter grid:
{'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}


In [6]:
# 매개변수로 사용할 분류모형, parameter 집합과 cross-validation의 k 등을 지정
grid_search = GridSearchCV(SVC(), param_grid, cv=5,
                          return_train_score=True)


grid_search.fit(newsgroups_train_tfidf, newsgroups_train.target)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [24]:
print("Test set score: {:.3f}".format(grid_search.score(newsgroups_train_tfidf, newsgroups_train.target)))
print("Test set score: {:.3f}".format(grid_search.score(newsgroups_test_tfidf, newsgroups_test.target)))

Test set score: 0.978
Test set score: 0.769


트레인셋은 약 98퍼센트의 예측률을 보이나 테스트셋은 약 77퍼센트의 예측률을 보인다.

In [47]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best score: {:.2f}".format(grid_search.best_score_)) 
# 사용한 파라미터 값 중에서 최선의 파라미터와 그에 대한 결과값을 나타낸다.

Best parameters: {'C': 10, 'gamma': 0.1}
Best score: 0.82


0.82가 최고 훈련 정확도이다.

In [52]:
from sklearn.metrics import accuracy_score


pred = grid_search.predict(newsgroups_test_tfidf)
print("Best test accuracy:\n{}".format(accuracy_score(newsgroups_test.target,pred)))  #최고 예측률 반환.

Best test accuracy:
0.7694013303769401


### Ridge regression

In [20]:
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier() #릿지 분류기 선언
ridge_clf.fit(newsgroups_train_tfidf, newsgroups_train.target) #학습
print('Train set score: {:.3f}'.format(ridge_clf.score(newsgroups_train_tfidf, newsgroups_train.target)))
print('Test set score: {:.3f}'.format(ridge_clf.score(newsgroups_test_tfidf, newsgroups_test.target)))

Train set score: 0.978
Test set score: 0.776


트레인셋은 97.8퍼센트, 테스트셋은 77.6퍼센트의 예측률을 보여주고 있다.

### Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression #sklearn이 제공하는 logistic regression을 사용
LR_clf = LogisticRegression() #분류기 선언
LR_clf.fit(newsgroups_train_tfidf, newsgroups_train.target) # train data를 이용하여 분류기를 학습

print('Train set score: {:.3f}'.format(LR_clf.score(newsgroups_train_tfidf, newsgroups_train.target))) 
print('Test set score: {:.3f}'.format(LR_clf.score(newsgroups_test_tfidf, newsgroups_test.target)))

Train set score: 0.940
Test set score: 0.733


트레인셋은 94퍼센트, 테스트셋은 73.3퍼센트의 예측률을 보여주고 있다.

### Naive Bayse Classifier

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 7000).fit(newsgroups_train.data) #max_feature를 제한하여 학습

X_train_cv = cv.transform(newsgroups_train.data) # train set을 변환
X_test_cv = cv.transform(newsgroups_test.data) # test set을 변환

In [39]:
from sklearn.naive_bayes import MultinomialNB #sklearn이 제공하는 MultinomialNB 를 사용
NB_clf = MultinomialNB() # Naive Bayse Classifier 분류기 선언
NB_clf.fit(X_train_cv, newsgroups_train.target)

print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, newsgroups_train.target))) 
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, newsgroups_test.target)))

Train set score: 0.884
Test set score: 0.764


counter vectorizer를 사용한 Naive Bayse Classifier의 예측률이다. max_features의 값이 올라갈수록 더 높은 예측률을 보여준다.

### 유사 텍스트 검색 및 추천

In [40]:
from sklearn.metrics.pairwise import cosine_similarity


X = cv.fit_transform(newsgroups_train.data) # 대상 텍스트를 counter vectorize.

new_review = "I've noticed that if you load a model" # 해당 문장과 가장 유사한 부분을 찾을 계획.
new_vec = cv.transform([new_review]) #문서를 count vector로 변환

sim_result = cosine_similarity(new_vec, X) # X와 기존 값들과의 similarity 계산

In [44]:
import numpy as np
np.argmax(sim_result[0]) 
# 가장 유사한 부분의 첫 번째 결과 위치 출력...인덱스

1371

In [45]:
print(newsgroups_train.data[1371])  # 1371번째 텍스트 출력.


But now you are contradicting yourself in a pretty massive way,
and I don't think you've even noticed.

In another part of this thread, you've been telling us that the
"goal" of a natural morality is what animals do to survive.

But suppose that your omniscient being told you that the long
term survival of humanity requires us to exterminate some 
other species, either terrestrial or alien.

Does that make it moral to do so?
